# [F21] The Security and Interpretability of Machine Learning
## Assignment 1
by Yaroslav Shumichenko



For this assignment, we will use a training pipeline tool, which allows to create and perform image
classification task with a single .yml configuration file. It is based on Pytorch Lightning:
https://github.com/PyTorchLightning/pytorch-lightning

The tool is developed in private github repository (request access if you need to look into the
source code):
https://github.com/Jud1cator/training-pipeline

### Training parameters description
We use kaggle version of CIFAR10 dataset (https://www.kaggle.com/swaroopkml/cifar10-pngs-in-folders)
which is in a Pytorch ImageFolder format. The 10% of train set is used for validation.

We use EfficientNetB0 model with pretrained weights from Ross Wightman's repository: https://github.com/rwightman/gen-efficientnet-pytorch

Other parameters can be found in config:
```
run_params:
  name: cifar10_effnetb0
  seed: 1

datamodule:
  name: ClassificationDataModule
  params:
    data_dir: "/home/judicator/data/cifar10/train"
    test_data_dir: "/home/judicator/data/cifar10/test"
    input_shape: [32, 32]
    val_split: 0.1
    batch_size: 128
    use_weighted_sampler: False
    pin_memory: True

task:
  name: ClassificationTask
  params:
    debug: False
    network:
      name: EfficientNetB0
      params:
        pretrained: True
    loss:
      name: CrossEntropyLoss
      params:
        is_weighted: False
    metrics:
      - name: F1Score
    optimizer:
      name: Adam
      params:
        lr: 0.001

callbacks:
  - name: ModelCheckpoint
    params:
      monitor: val_f1score
      mode: max
      verbose: True
  - name: EarlyStopping
    params:
      monitor: val_f1score
      mode: max
      verbose: True


trainer_params:
  max_epochs: 100
  gpus: 1

export_params:
  output_name: cifar10_effnetb0
  batch_size: 1
```

In [2]:
!python src/train.py -c cifar10_classification.yml

=> Warning: Pretrained model URL is empty, using random initialization.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type             | Params
------------------------------------------
0 | net  | GenEfficientNet  | 17.6 M
1 | loss | CrossEntropyLoss | 0     
------------------------------------------
17.6 M    Trainable params
0         Non-trainable params
17.6 M    Total params
70.266    Total estimated model params size (MB)
Validation sanity check:  50%|██████████          | 1/2 [00:00<00:00,  3.45it/s]/home/judicator/ev/training-pipeline/src/metrics/f_score.py:13: RuntimeWarning: invalid value encountered in true_divide
  precision = np.mean(np.nan_to_num(diag / confusion_matrix.sum(axis=0)))
Epoch 0:  90%|▉| 352/392 [00:38<00:04,  9.15it/s, loss=2.14, v_num=0, val_loss=2
Validating: 0it [00:00, ?it/s]
Validating:   0%|                                        | 0/40 [00:00<?, ?it/s]
Epoch 0:  90%|▉| 